In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 449 ms (started: 2021-08-28 16:19:45 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #2 Aug 28, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main c5f194e] 更新 #2 Aug 28, 2021
 2 files changed, 1287 insertions(+), 2 deletions(-)
 create mode 100644 "\345\246\202\344\275\225\347\273\204\347\273\207PYTORCH \345\210\260 LIGHTNING.ipynb"


To git@github.com:ustchope/pytorch_lightning_study.git
   944aae4..c5f194e  main -> main


time: 4.01 s (started: 2021-08-28 16:17:23 +08:00)


为了使您的代码能够与 Lightning 一起使用，以下是将 PyTorch 组织到 Lightning 中的方法

In [2]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl

time: 3.72 s (started: 2021-08-28 16:19:47 +08:00)


# 移动你的计算代码

移动模型架构并forward到您的闪电模块。

In [4]:
class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(28 * 28, 128)
        self.layer_2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        return x

time: 1.26 ms (started: 2021-08-28 16:20:04 +08:00)


# 移动优化器和调度器

将您的优化器移动到 configure_optimizers() 钩子。

In [6]:
class LitModel(pl.LightningModule):
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

time: 602 µs (started: 2021-08-28 16:21:47 +08:00)


# 找到训练循环“meat”

Lightning 为您自动化了大部分训练，即 epoch 和批次迭代，您需要保留的只是训练步骤逻辑。 这应该进入 training_step() 钩子（确保在这种情况下使用钩子参数，batch 和 batch_idx）：

In [5]:
class LitModel(pl.LightningModule):
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

time: 979 µs (started: 2021-08-28 16:21:43 +08:00)


# 找到 val 循环“meat”

要添加（可选）验证循环，请向 validation_step() 钩子添加逻辑（确保在这种情况下使用钩子参数 batch 和 batch_idx）。

In [7]:
class LitModel(pl.LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = F.cross_entropy(y_hat, y)
        return val_loss

time: 954 µs (started: 2021-08-28 16:22:50 +08:00)


> 自动调用 model.eval() 和 torch.no_grad() 进行验证

# 找到测试循环“meat”

要添加（可选）测试循环，请向 test_step() 钩子添加逻辑（确保在这种情况下使用钩子参数，batch 和 batch_idx）。

In [9]:
class LitModel(pl.LightningModule):
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

time: 905 µs (started: 2021-08-28 16:23:55 +08:00)


> 自动调用 model.eval() 和 torch.no_grad() 进行测试。

在您调用之前，不会使用测试循环。

In [ ]:
trainer.test()

> .test() 自动加载最佳检查点

# 删除任何 .cuda() 或 to.device() 调用

您的lightning模块可以在任何硬件上自动运行！